# SkyModel_cal

This notebook demonstrates a variant of SageCal algorithm, based on the paper:
Radio interferometric calibration with SAGE.

S Kazemi, S Yatawatta, S Zaroubi, P Lampropoulos, A G de Bruyn, L V E Koopmans, and J Noordam.

Monthly Notices of the Royal Astronomical Society, 2011 vol. 414 (2) pp. 1656-1666.

http://adsabs.harvard.edu/cgi-bin/nph-data_query?bibcode=2011MNRAS.414.1656K&link_type=EJOURNAL

This version allows images as well as discrete components


In [ ]:
%matplotlib inline

import os
import sys
sys.path.append(os.path.join('..','..'))

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from arl.skycomponent.operations import find_skycomponents, find_nearest_component
from arl.calibration.solvers import solve_gaintable
from arl.calibration.operations import apply_gaintable, create_gaintable_from_blockvisibility
from arl.data.data_models import Image
from arl.data.polarisation import PolarisationFrame
from arl.data.parameters import get_parameter
from arl.visibility.base import create_blockvisibility, copy_visibility
from arl.skycomponent.operations import create_skycomponent
from arl.image.deconvolution import restore_cube
from arl.skycomponent.operations import insert_skycomponent
from arl.image.operations import show_image, export_image_to_fits, qa_image, copy_image, create_empty_image_like
from arl.visibility.iterators import vis_timeslice_iter
from arl.visibility.coalesce import convert_visibility_to_blockvisibility
from arl.util.testing_support import create_named_configuration, create_low_test_beam, \
    simulate_gaintable, create_low_test_skycomponents_from_gleam
from arl.skycomponent.operations import apply_beam_to_skycomponent
from arl.imaging import create_image_from_visibility, advise_wide_field, predict_skycomponent_visibility
from arl.imaging.imaging_context import invert_function, predict_function

from arl.calibration.skymodel_cal import skymodel_cal_solve
from arl.calibration.skymodel_cal_delayed import create_skymodel_cal_solve_graph

from arl.graphs.dask_init import get_dask_Client
from arl.image.operations import export_image_to_fits

import logging

log = logging.getLogger()
log.setLevel(logging.INFO)
log.addHandler(logging.StreamHandler(sys.stdout))

We make the visibility. The parameter rmax determines the distance of the furthest antenna/stations used. All over parameters are determined from this number.

In [ ]:
nfreqwin = 1
ntimes = 1
rmax = 600
frequency = numpy.linspace(0.8e8, 1.2e8, nfreqwin)
if nfreqwin > 1:
    channel_bandwidth = numpy.array(nfreqwin * [frequency[1] - frequency[0]])
else:
    channel_bandwidth = [0.4e8]
times = numpy.linspace(-numpy.pi / 3.0, numpy.pi / 3.0, ntimes)

phasecentre = SkyCoord(
    ra=-70.0 * u.deg, dec=-60.0 * u.deg, frame='icrs', equinox='J2000')

lowcore = create_named_configuration('LOWBD2', rmax=rmax)

block_vis = create_blockvisibility(
    lowcore,
    times,
    frequency=frequency,
    channel_bandwidth=channel_bandwidth,
    weight=1.0,
    phasecentre=phasecentre,
    polarisation_frame=PolarisationFrame("stokesI"))

print(block_vis.vis.shape)

In [ ]:
wprojection_planes=1
advice=advise_wide_field(block_vis, guard_band_image=5.0, delA=0.02, wprojection_planes=wprojection_planes)

vis_slices = advice['vis_slices']
npixel=advice['npixels2']
cellsize=advice['cellsize']

Generate the model from the GLEAM catalog, including application of the primary beam.

In [ ]:
beam = create_image_from_visibility(
    block_vis,
    npixel=npixel,
    frequency=frequency,
    nchan=nfreqwin,
    cellsize=cellsize,
    phasecentre=phasecentre)

gleam_components = create_low_test_skycomponents_from_gleam(
    flux_limit=2.0,
    phasecentre=phasecentre,
    frequency=frequency,
    polarisation_frame=PolarisationFrame('stokesI'),
    radius=npixel * cellsize)

beam = create_low_test_beam(beam)
gleam_components = apply_beam_to_skycomponent(gleam_components, beam, flux_limit=0.5)
from matplotlib import pylab
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'


show_image(beam, components=gleam_components, cm='Greys', title='Primary beam plus original GLEAM components')
print("Number of components %d" % len(gleam_components))

Generate the template image

In [ ]:
model = create_image_from_visibility(block_vis, npixel=npixel, frequency=[numpy.average(frequency)], nchan=1,
    channel_bandwidth=[numpy.sum(channel_bandwidth)], cellsize=cellsize, phasecentre=phasecentre)

Create the model visibilities, applying a different gain table for each.

In [ ]:
corrupted_vis = copy_visibility(block_vis)
gt = create_gaintable_from_blockvisibility(block_vis, timeslice='auto')
isoplanatic_size = 0.005
for sc in gleam_components:
    component_vis = copy_visibility(block_vis, zero=True)
    separation=block_vis.phasecentre.separation(sc.direction).to('rad').value
    phase_error = numpy.power(separation, +1.8)/isoplanatic_size
    print("Offset from phase centre = %.3f radians, phase error = %.3f radians" % (separation, phase_error))
    gt = simulate_gaintable(gt, amplitude_error=0.0, phase_error=phase_error)
    component_vis = predict_skycomponent_visibility(component_vis, sc)
    component_vis = apply_gaintable(component_vis, gt)
    corrupted_vis.data['vis'][...]+=component_vis.data['vis'][...]
    
dirty, sumwt = invert_function(corrupted_vis, model, context='2d')
export_image_to_fits(dirty, 'skymodel_cal_before_dirty.fits')

Show the dirty image, along with the GLEAM components

In [ ]:
show_image(dirty, components=gleam_components, cm='Greys', title='Dirty image plus original components')
qa = qa_image(dirty, context='Dirty image plus original components')
print(qa)
plt.show()

Find the components above the threshold 10 times the median-abs of the dirty image

In [ ]:
qa = qa_image(dirty)
found_components= find_skycomponents(dirty, threshold=10.0*qa.data['medianabs'])
show_image(dirty, components=found_components, cm='Greys', title='Dirty image plus found components')
plt.show()

First do an isoplanatic selfcalibration using these components

In [ ]:
predicted_vis = copy_visibility(block_vis, zero=True)
predicted_vis = predict_skycomponent_visibility(predicted_vis, found_components)
gt = solve_gaintable(corrupted_vis, predicted_vis, phase_only=True, timescale='auto', seed=None)
corrupted_vis = apply_gaintable(corrupted_vis, gt, inverse=True)
dirty, sumwt = invert_function(corrupted_vis, model, context='2d')
export_image_to_fits(dirty, 'skymodel_cal_after_iso_dirty.fits')

qa = qa_image(dirty)
found_components= find_skycomponents(dirty, threshold=20.0*qa.data['medianabs'])
show_image(dirty, components=found_components, cm='Greys', title='Iso only: Dirty image plus found components')
plt.show()

Run skymodel_cal using dask

In [ ]:
client = get_dask_Client()
from arl.data.skymodel import SkyModel
from dask import delayed
skymodel_graph = [delayed(SkyModel, nout=1)(components=[cm]) for cm in found_components]
graph = create_skymodel_cal_solve_graph(corrupted_vis, skymodel_graph, niter=5, gain=0.25, tol=1e-8)
graph.visualize('skymodel_cal.svg')
graph = create_skymodel_cal_solve_graph(corrupted_vis, skymodel_graph, niter=30, gain=0.25, tol=1e-8)

calskymodel, residual_vis = client.compute(graph, sync=True)
client.close()

In [ ]:
skymodel_cal_components = [csm[0].components[0] for csm in calskymodel]

residual, sumwt = invert_function(residual_vis, model, context='2d')
show_image(residual, components=skymodel_cal_components, cm='Greys', title='Residual image after Sagecal')
plt.show()

psf, _ = invert_function(residual_vis, model, dopsf=True, context='2d')
component_image = copy_image(model)
component_image.data[...] = 0.0
component_image= insert_skycomponent(component_image, skymodel_cal_components)
restored = restore_cube(component_image, psf, residual)
export_image_to_fits(restored, 'skymodel_cal_after_noniso_dirty.fits')

print(qa_image(restored, context='Restored image after SageCal'))
show_image(restored, components=found_components, cm='Greys', title='Restored image after SageCal')
plt.show()